In [75]:
#dataframe에서 featuretools library를 이용하여 entity를 만들고 feature들을
#creation하거나, selection한다. -> featuretools를 이용하는 이유.

In [76]:
import pandas as pd
import numpy as np
import featuretools as ft

In [90]:
# Read in the data
clients = pd.read_csv('C:/Users/ksmin/data_Science/dataset/clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('C:/Users/ksmin/data_Science/dataset/loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('C:/Users/ksmin/data_Science/dataset/payments.csv', parse_dates = ['payment_date'])

In [91]:
clients.head()

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [92]:
#Create entity set
es = ft.EntitySet(id = 'clients')

In [93]:
 # Create an entity from the client dataframe
 # This dataframe already has an index and a time index
 # index=RDB primary key
 # time index= date_time feature
 es = es.entity_from_dataframe(entity_id = 'clients', 
                               dataframe = clients, 
                               index = 'client_id', 
                               time_index = 'joined')

In [94]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
  Relationships:
    No relationships

In [95]:
es = es.entity_from_dataframe(entity_id='loans', 
                               dataframe=loans,
                               index='loan_id', 
                               time_index='loan_end')

C:\Users\ksmin\anaconda3\Lib\site-packages\featuretools\utils\entity_utils.py:206: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(dropped_na, errors='raise')


In [96]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
  Relationships:
    No relationships

In [97]:
 # Create an entity from the payments dataframe
 # This does not have an index
 es = es.entity_from_dataframe(entity_id = 'payments', 
                               dataframe = payments,
                               variable_types = {'missed':
                                                 ft.variable_types.Categorical},
                               make_index = True,
                               index = 'payment_id',
                               time_index = 'payment_date')

In [98]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [84]:
es['payments']

Entity: payments
  Variables:
    payment_id (dtype: index)
    loan_id (dtype: numeric)
    payment_amount (dtype: numeric)
    payment_date (dtype: datetime_time_index)
    missed (dtype: categorical)
  Shape:
    (Rows: 3456, Columns: 5)

In [85]:
es['clients']

Entity: clients
  Variables:
    client_id (dtype: index)
    joined (dtype: datetime_time_index)
    income (dtype: numeric)
    credit_score (dtype: numeric)
  Shape:
    (Rows: 25, Columns: 4)

In [99]:
# Group loans by client id and calculate mean, max, min of loans
#client_id로 groupby를 한 다음 loan_amount의 mean,max,min 계산을 하여 출력한다.
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 
'max', 'min'])
#columns 생성한다.
stats.columns = ['mean_loan_amount', 'max_loan_amount', 
'min_loan_amount']
# Merge with the clients dataframe
stats = clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left')
stats.head(10)

,client_id,joined,income,credit_score,mean_loan_amount,max_loan_amount,min_loan_amount
0,46109,2002-04-16,172677,527,8951.600000,14049,559
1,49545,2007-11-14,104564,770,10289.300000,14971,3851
2,41480,2013-03-11,122607,585,7894.850000,14399,811
3,46180,2001-11-06,43851,562,7700.850000,14081,1607
4,25707,2006-10-06,211422,621,7963.950000,13913,1212
5,39505,2011-10-14,153873,610,7424.050000,14575,904
6,32726,2006-05-01,235705,730,6633.263158,14802,851
7,35089,2010-03-01,131176,771,6939.200000,13194,773
8,35214,2003-08-08,95849,696,7173.555556,14767,667
9,48177,2008-06-09,190632,769,7424.368421,14740,659


In [100]:
# Create a relationship between clients and loans
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])
# Add the relationship to the entity set
es= es.add_relationship(r_client_previous)
# Relationship between previous loans and previous payments
r_payments = ft.Relationship(es['loans']['loan_id'],
                             es['payments']['loan_id'])
# Add the relationship to the entity set
es= es.add_relationship(r_payments)
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

In [120]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['sum'],
                                 trans_primitives=[])

C:\Users\ksmin\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:698: FutureWarning: The provided callable <function sum at 0x00000213A282BF60> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  observed=True, sort=False).agg(to_agg)
C:\Users\ksmin\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:698: FutureWarning: The provided callable <function sum at 0x00000213A282BF60> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  observed=True, sort=False).agg(to_agg)


In [121]:
features

,income,credit_score,SUM(loans.loan_amount),SUM(loans.repaid),SUM(loans.rate),SUM(payments.payment_amount),SUM(payments.loans.repaid),SUM(payments.loans.rate),SUM(payments.loans.loan_amount)
client_id,,,,,,,,,
42320,229481,563,105931,9,36.86,122578,70,302.84,843367
39384,191204,617,149444,12,67.23,174270,84,499.12,1161741
26945,214516,806,106889,6,42.83,124261,38,330.13,771053
41472,152214,638,120173,8,63.70,118553,56,435.36,784731
46180,43851,562,154017,10,70.05,176796,74,578.43,1142666
46109,172677,527,179032,11,63.05,193954,76,450.55,1199661
32885,58955,642,148806,8,36.54,160597,63,244.97,1086206
29841,38354,523,176634,10,62.01,215915,84,494.77,1479162
38537,127183,643,152768,10,40.62,164904,72,318.07,1086422


In [89]:
primitives = ft.primitives.list_primitives()
print(primitives)

                     name         type  dask_compatible  \
0                     all  aggregation             True   
1        avg_time_between  aggregation            False   
2                    last  aggregation            False   
3                    mode  aggregation            False   
4                     sum  aggregation             True   
..                    ...          ...              ...   
74       subtract_numeric    transform             True   
75    greater_than_scalar    transform             True   
76         modulo_numeric    transform             True   
77                is_null    transform             True   
78  divide_numeric_scalar    transform             True   

                                          description  
0      Calculates if all values are 'True' in a list.  
1   Computes the average number of seconds between...  
2                Determines the last value in a list.  
3        Determines the most commonly repeated value.  
4      Calc